# Tekstikolletsioonid ja nende kasutamine

Tekstikollekstsioonide (*korpuste*)  töötlemiseks on olemas kaks põhimõttelist võimalust:

* tekstid on salvestatud failidena (*pickle*),
* tekstid on salvestatud andmebaasi objektidena.

Andmebaasi kasutamise eeliseks on parem otsitavus ning võimalus kasutada olemasolevaid algoritme.

* EstNLTK 1.4 teek kasutab [ElasticSearch](https://www.elastic.co/products/elasticsearch) otsingumootorit teksikollektsioonide salvestamiseks. 
* EstNLTK 1.6 teek kasutab [PostgreSQL](https://www.postgresql.org) andmebaasi teksikollektsioonide salvestamiseks. 

PostgreSQL eeliseks:

* stabiilne API,
* ennustatav resursi kulu,
* parem skaleeruvus praktiliste andmemahtude juures,
* lihtsam integreeritavus olemasolevasse it-taristusse.

### Tüüpilised sammud tekstikollektsioonide töötamisel 
* Tekstikollektsiooni loomine
* Tekstikollektsiooni annoteerimine
* Tekstikollektsioonidest otsimine


## 0. Ettevalmistavad sammud 

Selleks et Jupyteriga oleks lihtsam töötada tuleks soovitusi muuta agresiivsemaks (*autocomplete*).

In [1]:
%config IPCompleter.greedy=True

See võimaldab meil *Tab*-iga küsida objekti meetodeid ning *Shift+Tab*-iga küsida funktsioonide dokumentatsiooni.

## 1. Tekstikollektsioonide loomine failidest

Selleks et oleks selge, mis mooduleid konkreetsetes sammudes kasutatakse, impordime moodulid vajaduse põhiselt iga sammu ees.

### Sisendtektide uurimine

Järgnevas on meie eesmärk uurida raadiosaadete automaat-transkriptsiooni abil saadud materjale. Vastavad failid asuvad kataloogis `data/kpt` ning on UTF-8 kodeeringus. Nende lugemiseks on vaja fail õiges kodeeringus avada.
Üldiselt pole UTF-8 avamiseks vaja erilisi liigutusi teha, aga üldiselt kasutatakse selleks `codec` teeki.  

In [4]:
import codecs
f1 = codecs.open("data/kpt/2019-09-07.txt", "r", "utf-8")
raw_text = f1.read()
print(raw_text[:300])

K01: Kuku raadios välja öeldud seisukohad ei pea ühtima Kuku raadio seisukohtadega. Te kuulate Kuku raadiot?
-:
K02: Breivik tahtis proovida, mismoodi õhu lenn Sid raisakotkas kohe appi pahenduse, kui õhusegasse Aalto tegi, kaval linn. Natuurist haaras, pärdik öeldes kuuleb. Palun lenda sirgelt. Pal


Teksti vaadates on selge, et iga repliik algab kõneleja nimega, millele järgneb repliik. 
Selline struktuur on väha sagedane ka ametlike dokumentide või tootekirjelduste korral. 
Ikka kasutatakse dokumendi eri osade eraldamiseks pealkirju või muid korduvaid struktuurielemente. 
Tavaliselt on kõige lihtsam selliseid struktuurielemente tuvastada regulaaravaldiste abil.  

## Teksti esmane segmenteerimine

Esimese asjana tuleks tekstist leida üles kõnetuvastaja poolt pandud rääkija tähis või nimi. Selleks kasutame kahte märgendajat:
* ```ŖegexTagger``` abil märgime peale rääkija
* ```TextSegmentsTagger``` abil märgime peale rääkija kõneldud laused 

### Kõnelejale vastav regulaaravaldis

Kuna kõnetuvastuse väljund on selgete struktuuriga, siis on vastava regulaaravaldise leidmine üsna lihtne.
Sama olukord on ka keerukamate poolstruktureeritud dokumentide segmenteerimisel. Tüüpiliselt on üsna lihtne leida korduvaid mustreid, mis defineerivad erinevate tekstide osasid (lõikude nummerdus, kuupäevaline kirje päis, jms).

Igal juhul on vastava regulaaravaldise tuletamiseks ja testimiseks vaja võtta illustreerivad näited. Nendest saab hiljem luua ka testi, mille abil tulevikus tagada funktsionaalsuse säilimine mustrite täiendamisel.

In [6]:
import re

In [7]:
test_text =  'K01: Kuku...\n-:\nArtur Talvik: Tere, siin on...'

Kuna regulaaravaldistest arusaamine on keerukas, siis teeme seda sammhaaval. 
Selleks, et mitte takerduda paosümbolite (*escape symbols*) rägastikku, on mõistlik kasutada `r'sõnesid'`.
Nii ei ole vaja regulaaravalidiste paosümboleid mitmekordselt pagendada.

In [14]:
print(re.findall(r'^.',test_text))
print(re.findall(r'((^|\n).)',test_text))
print(re.findall(r'((^|\n).*:)',test_text))

['K']
[('K', ''), ('\n-', '\n'), ('\nA', '\n')]
[('K01:', ''), ('\n-:', '\n'), ('\nArtur Talvik:', '\n')]


Et edaspidi oleks lihtsam, tuleks regulaaravaldise osadele anda nimed:
* ```name``` - kõneleja nimi,
* ```span``` - kõneleja fikseeriv tekstifragment.

In [16]:
pattern = r'((^|\n)(?P<span>(?P<name>.*): ?))'
print([match.group('name') for match in re.finditer(pattern, test_text)])
print([match.group('span') for match in re.finditer(pattern, test_text)])
print([match.group() for match in re.finditer(pattern, test_text)])

['K01', '-', 'Artur Talvik']
['K01: ', '-:', 'Artur Talvik: ']
['K01: ', '\n-:', '\nArtur Talvik: ']


### RegexTagger mustrisõnastik

Märgendaja ```RegexTagger``` defineerimiseks on peale mustrite vaja veel spetsifitseerida mitu olulist asja:
* ```_regex_pattern_``` - regulaaravaldis,
* ```_group_``` - *spani* defineeriv regulaaravaldise grupp,
* ```_priority_``` - mustri prioriteet konfliktide korral,
* ```_validator_``` - validaator valepositiivsete vastete eemldamiseks.

Neist esimesed kaks on alati vajalikud ning ülejäänud on kasulikud enamikel juhtudel. Meie näites on meil märgendamiseks vajalik vaid üks muster. 

In [18]:
header_voc_1 = \
[{
    '_regex_pattern_': pattern,
    '_group_': 'span',
    '_priority_': 1,
}]

Lisaks on mõistlik defineerida ka *spanidele* vastavad annotatsiooni atribuudid. Meie näites siis kõneleja isik. Selles tuleb  anda ette funktsioon, mis võttab sisse kogu regulaaravalidsele vastava ```Match``` objekti ja tuletab sellest atribuudi väärtuse.  

In [19]:
header_voc_2 = \
[{
    '_regex_pattern_': pattern,
    '_group_': 'span',
    '_priority_': 1,
    'person': lambda m: m.group('name') 
}]

In [9]:
header_voc_3 = \
[{
    '_regex_pattern_': pattern,
    '_group_': 'span',
    '_priority_': 1,
    '_validator_': lambda m: m.group('span') != '-:', 
    'person': lambda m: m.group('name') 
}]

### RegexTagger loomine

Märgendaja `RegexTagger` loomisel on tarvis alati määrata peamised parameetrid:

* `vocabulary` - regulaaravaldiste mustrite sõnastik,
* `output_layer` - väljundkihi nimi,
* `output_attributes` - väljundkihi tellitavate atribuutide nimed.

Lisaks saab mängida erinevate täpishäälestust võimaldavate parameetritega:
* `ambiguous` - kas leitud märgendused võib olla mitu annotatsiooni,
* `conflict_resolving_strategy` - mida teha kui regulaaravalistele vastavad fragmendid on ülekattes 
* `overlapped, ignore_case` - lisaargumendid argumendid `re.finditer` funktsiooni häälestamiseks

Üldiselt on täpishäälestuse parameetreid vaja vaid siis, kui mustrisõnastik käitub ootamatult ning on tarvis aru saada, mis läks valesti.  

In [20]:
from estnltk import Text
from estnltk.taggers import RegexTagger

Märgendaja ei pea annoteerima valitud teksti 

In [31]:
tagger = RegexTagger(vocabulary = header_voc_1, output_layer = 'headers') 
text = tagger.tag(Text(raw_text))
display(text.headers[:5])

Layer(name='headers', attributes=(), spans=SL[Span('K01: ', [{}]),
Span('-:', [{}]),
Span('K02: ', [{}]),
Span('-:', [{}]),
Span('K06: ', [{}])])

Selleks, et et märgendaja annoteeriks teksti peab:

* mustrisõnastikus olema reegel atribuudi arvutamiseks,
* vastav atribuut peab olema tellitud väljundkihti.

In [36]:
# Atribuudid pole tellitud
tagger = RegexTagger(vocabulary = header_voc_2, output_layer = 'headers') 
text = tagger.tag(Text(raw_text))
display(text.headers[:3])

# Atribuut person on tellitud 
tagger = RegexTagger(
    vocabulary = header_voc_2, 
    output_layer = 'headers',
    output_attributes = ['person']
) 

text = tagger.tag(Text(raw_text))
display(text.headers[:3])

Layer(name='headers', attributes=(), spans=SL[Span('K01: ', [{}]),
Span('-:', [{}]),
Span('K02: ', [{}])])

Layer(name='headers', attributes=('person',), spans=SL[Span('K01: ', [{'person': 'K01'}]),
Span('-:', [{'person': '-'}]),
Span('K02: ', [{'person': 'K02'}])])

### Teksti segmenteerimine erinevate inimeste kõneks

Järgmiseks loomulikuks sammuks on repliikide eraldamine üldtekstist. Selle käigus on mõistlik kõneleja isik panna eraldi atribuudiks.
Jällegi on tegemist standardse dokumendi struktureerimise ülesandega, mille käigus jagatakse põhitekst päiste (*header*) järgi osadeks. 

Selle jaoks on EstNLTK teegis olemas `TextSegmentsTagger` märgendaja, mille loomisel on tarvis määrata peamised parameetrid:

* `input_layer` - päiselementide kiht
* `output_layer` - väljundkiht
* `output_attributes` - väljundkihti tellitavate atribuutide nimekiri

In [37]:
from estnltk.taggers import TextSegmentsTagger

In [41]:
tagger = TextSegmentsTagger(
    input_layer = 'headers', 
    output_layer = 'lines',
    output_attributes = ['person'])

tagger.tag(text)
text.lines[:5]

Layer(name='lines', attributes=('person',), spans=SL[Span('Kuku raadios välja öeldud seisukohad ei pea ühtima Kuku raadio seisukohtadega. Te kuulate Kuku raadiot?\n', [{'person': None}]),
Span('\n', [{'person': None}]),
Span('Breivik tahtis proovida, mismoodi õhu lenn Sid raisakotkas kohe appi pahenduse, kui õhusegasse Aalto tegi, kaval linn. Natuurist haaras, pärdik öeldes kuuleb. Palun lenda sirgelt. Palun lenda kõrge. Võtta asja kerge, tappa maha, ära päädhabi.\n', [{'person': None}]),
Span('\n', [{'person': None}]),
Span('Ta oli liiga uudis ainult koduvabariigist.\n', [{'person': None}])])

Saadud tulemus on vigane:

* kuigi me oleme tellinud väljundkihti kõneleja nime ei jõua selle väärtus atribuutide hulka;
* repliikide hulka on sattunud palju tühje tekste

Nende probleemide lahendamiseks tuleb meil määrata märgendaja `TextSegmentsTagger` parameetrid:

* `decorator` - funktsioon tellitud attribuutide väärtuste arvutamiseks,
* `validator` - funktsioon päiste täiendavaks valideerimiseks.

Viimaks saab parameetriga 

* `include_header` - määrata kas päiselement kuulub tekstiosa koosseisu või mitte. 

Tüüpiliselt on mõistlik päisest mõelda kui tekstiosale vastavast metainfost ja seega see ei peaks olema teksti osa. 

In [42]:
tagger = TextSegmentsTagger(
    input_layer = 'headers', 
    output_layer = 'lines',
    output_attributes = ['person'],
    decorator = lambda header_span: {'person': header_span['person']},
    validator = lambda header_span: header_span['person'] != '-' 
)
if 'lines' in text.layers:
    del text.lines
tagger.tag(text)
display(text.lines[:5])
display(text.lines[0])

Layer(name='lines', attributes=('person',), spans=SL[Span('Kuku raadios välja öeldud seisukohad ei pea ühtima Kuku raadio seisukohtadega. Te kuulate Kuku raadiot?\n-:\n', [{'person': 'K01'}]),
Span('Breivik tahtis proovida, mismoodi õhu lenn Sid raisakotkas kohe appi pahenduse, kui õhusegasse Aalto tegi, kaval linn. Natuurist haaras, pärdik öeldes kuuleb. Palun lenda sirgelt. Palun lenda kõrge. Võtta asja kerge, tappa maha, ära päädhabi.\n-:\n', [{'person': 'K02'}]),
Span('Ta oli liiga uudis ainult koduvabariigist.\n', [{'person': 'K06'}]),
Span('Tere, siin on Keskpäevatund ja Kuku raadio, Tallinna stuudios Urmas Jaagant, Ainar Ruussaar ja Priit Hõbemägi. Mart Sander laulis lõbusa laulukese. Palun lenda sirgelt ja see laul on meie poolt tervituseks Gunnar kobinile. Kes kelle kohta hiljuti saime teada, et ta loobus Nordika juhi ametikohast, nimelt ta pidi Kobin alustama teisel septembril tordika juhina, aga ootamatult selgus, et ta on juba suvel oma ametikohast loobunud ja tõi ühe põhjusena välja selle, et ühiskonnas tekkinud suur diskussioon tema töötasu üle, mis oli siis viisteist tuhat bruto ühes kuus ei ole vastavuses tema tegeliku töö panusena. Et seda tema lauset oskan mina kahte moodi analüüsida, üks on see, et liiga, et talle makstakse liiga palju raha ja tema panuseks väikeseks, aga teisest küljest võiks mõelda ka niimoodi, et ta ütleb, et selle panuse eest, mis tema suudaks ettevõtte arengusse panna, on seda raha liiga vähe. Kuidas see asi tegelikult on, eks teab ta ise.\n', [{'person': 'Artur Talvik'}]),
Span('Ma arvan, et asi on lihtne nagu tavapäraselt, et Gunnar Kobin ilmselt läks süsteemi mille esiesmapilgul seal tundmaõppimisel said aru, et tema natuur, tema eesmärgid ja see süsteem, kuhu ta sattus ei klapi päris hästi kokku.\n', [{'person': 'Ainar Ruussaar'}])])

text,person
Kuku raadios välja öeldud seisukohad ei pea ühtima Kuku raadio seisukohtadega. Te kuulate Kuku raadiot?\n-:\n,K01


Saadud tulemus on ikkagi vigane, kuna päiselementide ignoreerimine suureneab vaid tekstilõikude pikkust ning ei jäta vastatatele päiselementidele vastavaid tekstiosasid välja. Seega tuleb esmalt need tekstiosad siiski sisse jätta. 

In [43]:
tagger = TextSegmentsTagger(
    input_layer = 'headers', 
    output_layer = 'lines',
    output_attributes = ['person'],
    decorator = lambda header_span: {'person': header_span['person']}
)
if 'lines' in text.layers:
    del text.lines
tagger.tag(text)
display(text.lines[:5])
display(text.lines[0])

Layer(name='lines', attributes=('person',), spans=SL[Span('Kuku raadios välja öeldud seisukohad ei pea ühtima Kuku raadio seisukohtadega. Te kuulate Kuku raadiot?\n', [{'person': 'K01'}]),
Span('\n', [{'person': '-'}]),
Span('Breivik tahtis proovida, mismoodi õhu lenn Sid raisakotkas kohe appi pahenduse, kui õhusegasse Aalto tegi, kaval linn. Natuurist haaras, pärdik öeldes kuuleb. Palun lenda sirgelt. Palun lenda kõrge. Võtta asja kerge, tappa maha, ära päädhabi.\n', [{'person': 'K02'}]),
Span('\n', [{'person': '-'}]),
Span('Ta oli liiga uudis ainult koduvabariigist.\n', [{'person': 'K06'}])])

text,person
Kuku raadios välja öeldud seisukohad ei pea ühtima Kuku raadio seisukohtadega. Te kuulate Kuku raadiot?\n,K01


### Tükelduse esmane valideerimine
??

In [44]:
from pandas import DataFrame

In [48]:
text.lines.groupby(['person']).count

{('K01',): 1,
 ('-',): 3,
 ('K02',): 1,
 ('K06',): 3,
 ('Artur Talvik',): 21,
 ('Ainar Ruussaar',): 16,
 ('K07',): 13,
 ('Ignar Fjuk',): 2}

In [55]:
DataFrame.from_dict(text.lines.groupby(['person']).count, orient='index', columns=['repliike'])

,repliike
"(K01,)",1
"(-,)",3
"(K02,)",1
"(K06,)",3
"(Artur Talvik,)",21
"(Ainar Ruussaar,)",16
"(K07,)",13
"(Ignar Fjuk,)",2


In [56]:
#from estnltk import EnvelopingBaseSpan

In [90]:
text.lines.groupby(['person']).aggregate(func = lambda spans: [(s.start, s.end) for s in spans])

{('K01',): [(5, 109)],
 ('-',): [(111, 112), (362, 363), (42112, 42112)],
 ('K02',): [(117, 360)],
 ('K06',): [(368, 411), (22876, 22930), (33280, 33323)],
 ('Artur Talvik',): [(425, 1377),
  (1858, 1960),
  (2464, 3392),
  (3911, 4675),
  (5109, 5441),
  (5833, 7970),
  (10867, 12410),
  (12798, 12992),
  (15067, 17077),
  (17425, 17512),
  (20014, 22844),
  (22944, 23597),
  (24164, 24315),
  (28038, 29361),
  (30160, 30213),
  (30447, 31030),
  (32163, 32346),
  (33189, 33248),
  (33337, 35311),
  (37663, 37826),
  (39986, 42110)],
 ('Ainar Ruussaar',): [(1393, 1618),
  (3408, 3897),
  (5457, 5819),
  (7986, 9325),
  (12426, 12784),
  (13008, 13649),
  (17093, 17411),
  (17528, 18562),
  (23613, 24150),
  (27696, 28024),
  (31046, 31649),
  (32362, 32603),
  (35327, 36059),
  (36672, 37649),
  (37842, 38081),
  (39224, 39972)],
 ('K07',): [(1623, 1844),
  (1965, 2450),
  (4680, 5095),
  (9330, 10853),
  (13654, 15053),
  (18567, 20000),
  (24320, 27680),
  (29366, 30146),
  (30218, 

In [81]:
text.lines.groupby(['person']).groups[('-',)]

[Span('\n', [{'person': '-'}]),
 Span('\n', [{'person': '-'}]),
 Span('', [{'person': '-'}])]

In [95]:
from string import strip

ImportError: cannot import name 'strip'

In [105]:
for span in text.lines:
    line = span.text.strip()
    if line == '':
        continue
    display(Text(line))
    break


text
Kuku raadios välja öeldud seisukohad ei pea ühtima Kuku raadio seisukohtadega. Te kuulate Kuku raadiot?


In [106]:
from estnltk.storage.postgres import PostgresStorage, create_schema, delete_schema
from estnltk.storage.postgres import JsonbTextQuery, JsonbLayerQuery, WhereClause, SubstringQuery

TypeError: create_schema() takes 1 positional argument but 2 were given

In [112]:
storage = PostgresStorage(host='127.0.0.1',
                          port=5432,
                          dbname='ekt',
                          user='swen',
                          password='kala',
                          schema='media_analysis',
                          role=None,
                          temporary=False)

INFO:storage.py:42: connecting to host: '127.0.0.1', port: 5432, dbname: 'ekt', user: 'swen'
INFO:storage.py:58: schema: 'media_analysis', temporary: False, role: 'swen'


In [114]:
create_schema(storage)

DuplicateSchema: schema "media_analysis" already exists


In [210]:
storage['kpt'].meta = {'date':'datetime', 'person':'str'}

In [211]:
storage['kpt'].create('Keskpäevatunni saadete automaatne transcriptsioon')#, meta={'date':'datetime', 'person':'str'})

INFO:collection.py:107: new empty collection 'kpt' created


,data type
date,timestamp without time zone
person,text


In [212]:
storage['kpt'].column_names += list(storage['kpt'].meta.keys())

In [213]:
storage['kpt'].column_names 

['id', 'data', 'date', 'person']

In [209]:
storage['kpt'].delete()

In [123]:
from datetime import datetime

In [124]:
datetime.strptime('2019-09-07', '%Y-%m-%d')

datetime.datetime(2019, 9, 7, 0, 0)

In [142]:
header_voc_2 = \
[{
    '_regex_pattern_': pattern,
    '_group_': 'span',
    '_priority_': 1,
    'person': lambda m: m.group('name') 
}]

header_tagger = RegexTagger(
    vocabulary = header_voc_2, 
    output_layer = 'headers', 
    output_attributes = ['person'])

In [143]:
segmenter = TextSegmentsTagger(
    input_layer = 'headers', 
    output_layer = 'lines',
    output_attributes = ['person'],
    decorator = lambda span: {'person': span['person']},
)

In [ ]:
import os

In [214]:
collection = storage['kpt']
directory = os.fsencode('data')

with collection.insert() as collection_insert:

    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename == 'readme.md':
            continue
        
        date = datetime.strptime(re.search(r'(?P<date>.*)\.txt', filename).group('date'),'%Y-%m-%d')
        file = codecs.open("data/2019-09-07.txt", "r", "utf-8")
        text = segmenter(header_tagger(Text(file.read())))
    
        for span in text.lines:
            line = span.text.strip()
            if line == '':
                continue
                
            line_text = Text(line)
            #line_text.analyse('segmentation')
            line_text.tag_layer(['compound_tokens', 'words', 'paragraphs'])
            meta_data = {'date': date, 'person': span['person']}
            collection_insert(text=line_text, meta_data=meta_data)

INFO:collection.py:325: inserted 455 texts into the collection 'kpt'


In [195]:
from estnltk.taggers import WordTagger
from estnltk.taggers import VabamorfTagger 


In [215]:
collection[0]

Text(text='Kuku raadios välja öeldud seisukohad ei pea ühtima Kuku raadio seisukohtadega. Te kuulate Kuku raadiot?')

In [220]:
vabamorf = VabamorfTagger(disambiguate = True, output_layer = 'morph_analysis')



In [221]:
tmp = vabamorf(collection[0])

In [222]:
tmp.morph_analysis

Layer(name='morph_analysis', attributes=('normalized_text', 'lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech'), spans=SL[Span('Kuku', [{'normalized_text': 'Kuku', 'lemma': 'Kuku', 'root': 'Kuku', 'root_tokens': ['Kuku'], 'ending': '0', 'clitic': '', 'form': 'sg g', 'partofspeech': 'H'}]),
Span('raadios', [{'normalized_text': 'raadios', 'lemma': 'raadio', 'root': 'raadio', 'root_tokens': ['raadio'], 'ending': 's', 'clitic': '', 'form': 'sg in', 'partofspeech': 'S'}]),
Span('välja', [{'normalized_text': 'välja', 'lemma': 'välja', 'root': 'välja', 'root_tokens': ['välja'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'D'}]),
Span('öeldud', [{'normalized_text': 'öeldud', 'lemma': 'öeldud', 'root': 'öeldud', 'root_tokens': ['öeldud'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'A'}, {'normalized_text': 'öeldud', 'lemma': 'öeldud', 'root': 'öeldud', 'root_tokens': ['öeldud'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'A'}, {'normalized_text': 'öeldud', 'lemma': 'öeldud', 'root': 'öeldud', 'root_tokens': ['öeldud'], 'ending': 'd', 'clitic': '', 'form': 'pl n', 'partofspeech': 'A'}, {'normalized_text': 'öeldud', 'lemma': 'ütlema', 'root': 'ütle', 'root_tokens': ['ütle'], 'ending': 'dud', 'clitic': '', 'form': 'tud', 'partofspeech': 'V'}]),
Span('seisukohad', [{'normalized_text': 'seisukohad', 'lemma': 'seisukoht', 'root': 'seisu_koht', 'root_tokens': ['seisu', 'koht'], 'ending': 'd', 'clitic': '', 'form': 'pl n', 'partofspeech': 'S'}]),
Span('ei', [{'normalized_text': 'ei', 'lemma': 'ei', 'root': 'ei', 'root_tokens': ['ei'], 'ending': '0', 'clitic': '', 'form': 'neg', 'partofspeech': 'V'}]),
Span('pea', [{'normalized_text': 'pea', 'lemma': 'pidama', 'root': 'pida', 'root_tokens': ['pida'], 'ending': '0', 'clitic': '', 'form': 'o', 'partofspeech': 'V'}]),
Span('ühtima', [{'normalized_text': 'ühtima', 'lemma': 'ühtima', 'root': 'ühti', 'root_tokens': ['ühti'], 'ending': 'ma', 'clitic': '', 'form': 'ma', 'partofspeech': 'V'}]),
Span('Kuku', [{'normalized_text': 'Kuku', 'lemma': 'kuku', 'root': 'kuku', 'root_tokens': ['kuku'], 'ending': '0', 'clitic': '', 'form': 'sg g', 'partofspeech': 'A'}]),
Span('raadio', [{'normalized_text': 'raadio', 'lemma': 'raadio', 'root': 'raadio', 'root_tokens': ['raadio'], 'ending': '0', 'clitic': '', 'form': 'sg g', 'partofspeech': 'S'}]),
Span('seisukohtadega', [{'normalized_text': 'seisukohtadega', 'lemma': 'seisukoht', 'root': 'seisu_koht', 'root_tokens': ['seisu', 'koht'], 'ending': 'dega', 'clitic': '', 'form': 'pl kom', 'partofspeech': 'S'}]),
Span('.', [{'normalized_text': '.', 'lemma': '.', 'root': '.', 'root_tokens': ['.'], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z'}]),
Span('Te', [{'normalized_text': 'Te', 'lemma': 'sina', 'root': 'sina', 'root_tokens': ['sina'], 'ending': '0', 'clitic': '', 'form': 'pl n', 'partofspeech': 'P'}]),
Span('kuulate', [{'normalized_text': 'kuulate', 'lemma': 'kuulama', 'root': 'kuula', 'root_tokens': ['kuula'], 'ending': 'te', 'clitic': '', 'form': 'te', 'partofspeech': 'V'}]),
Span('Kuku', [{'normalized_text': 'Kuku', 'lemma': 'kuku', 'root': 'kuku', 'root_tokens': ['kuku'], 'ending': '0', 'clitic': '', 'form': 'sg g', 'partofspeech': 'A'}]),
Span('raadiot', [{'normalized_text': 'raadiot', 'lemma': 'raadio', 'root': 'raadio', 'root_tokens': ['raadio'], 'ending': 't', 'clitic': '', 'form': 'sg p', 'partofspeech': 'S'}]),
Span('?', [{'normalized_text': '?', 'lemma': '?', 'root': '?', 'root_tokens': ['?'], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z'}])])

In [224]:
collection.create_layer(tagger=vabamorf)

INFO:collection.py:817: collection: 'kpt'
INFO:collection.py:836: preparing to create a new layer: 'morph_analysis'
INFO:collection.py:869: inserting data into the 'morph_analysis' layer table
INFO:collection.py:904: layer created: 'morph_analysis'


In [225]:
from estnltk.storage.postgres import SubstringQuery, JsonbTextQuery, JsonbLayerQuery

In [226]:
for key, text in collection.select(SubstringQuery('kuku')):
    print(text)

Text(text='Aga praegu peab vist Nordikat iseloomustada nii nagu aastaid tagasi. Noorem laps oli veel väike ja ma käisin temaga koos kinos vaatamas multikaid siis üks multikas Madagaskar, selle üks stseen, kus Pidviinidest lendurid teatasid reisijatele, et meil on teile hea uudis ja halb uudis, hea uudis on see, et me maandume, halb uudis on see, et me kukume, nagu kyll.')
Text(text='Keskpäevatund jätkab Urmas Jaagant, Ainar Ruussaar ja Priit Hõbemägi Tallinna stuudios. Brexiti on Ühendkuningriigi lahkumine Euroopa Liidust või keegi veel seda ei tea, aga need, kes seda teavad, need on võinud selle nädala jooksul jälgida maailma parimad TV-d raamat, mis on siis vaadeldav nii BBC koduleheküljelt kui Briti parlamendi koduleheküljelt, kui ka kõikidest suurematest maailma infoportaalidest, mis räägivad ja mis näitab otseülekandes seda, kuidas Briti parlament vaidleb selle üle, kas britid peaksid kukkuma kolmekümne esimesel oktoobril kolksti Euroopa Liidust välja ilma igasuguste lepingute või

In [229]:
collection.select(SubstringQuery('kuku')).head(1)

[(13,
  Text(text='Aga praegu peab vist Nordikat iseloomustada nii nagu aastaid tagasi. Noorem laps oli veel väike ja ma käisin temaga koos kinos vaatamas multikaid siis üks multikas Madagaskar, selle üks stseen, kus Pidviinidest lendurid teatasid reisijatele, et meil on teile hea uudis ja halb uudis, hea uudis on see, et me maandume, halb uudis on see, et me kukume, nagu kyll.'))]

In [239]:
JsonbTextQuery('words', text='Kuku').eval(storage, 'kpt')

Composed([Composed([Identifier('media_analysis'), SQL('.'), Identifier('kpt')]), SQL('."data"->\'layers\' @> \'[{'), SQL('"name": '), Identifier('words'), SQL(', "spans": [{'), SQL('"annotations": ['), SQL('{"text": "Kuku"}'), SQL(']}'), SQL(']}'), SQL("]'")])

In [234]:
collection.select(JsonbTextQuery('words', text='Kuku')).head() 

[]

In [249]:
collection.selected_layers =['morph_analysis']
collection.selected_layers

['words', 'morph_analysis']

In [257]:
for key, text in collection.select(layers=['words', 'morph_analysis'],layer_query ={'morph_analysis': JsonbLayerQuery('morph_analysis', lemma='Kuku')}):
    display(text)
    text.words
    break

Text(text='Kuku raadios välja öeldud seisukohad ei pea ühtima Kuku raadio seisukohtadega. Te kuulate Kuku raadiot?')

In [232]:
collection[0].words

Layer(name='words', attributes=('normalized_form',), spans=SL[Span('Kuku', [{'normalized_form': None}]),
Span('raadios', [{'normalized_form': None}]),
Span('välja', [{'normalized_form': None}]),
Span('öeldud', [{'normalized_form': None}]),
Span('seisukohad', [{'normalized_form': None}]),
Span('ei', [{'normalized_form': None}]),
Span('pea', [{'normalized_form': None}]),
Span('ühtima', [{'normalized_form': None}]),
Span('Kuku', [{'normalized_form': None}]),
Span('raadio', [{'normalized_form': None}]),
Span('seisukohtadega', [{'normalized_form': None}]),
Span('.', [{'normalized_form': None}]),
Span('Te', [{'normalized_form': None}]),
Span('kuulate', [{'normalized_form': None}]),
Span('Kuku', [{'normalized_form': None}]),
Span('raadiot', [{'normalized_form': None}]),
Span('?', [{'normalized_form': None}])])

In [231]:
collection.select(JsonbTextQuery('morph_analysis', lemma='kuku')).head() 

[]